In [1]:
# For Sam

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

loading the dataset

In [3]:
dataset = pd.read_csv("Train.csv")
finaltest = pd.read_csv("Test.csv")
riders = pd.read_csv('Riders.csv')
sample = pd.read_csv('SampleSubmission.csv')

Checking it out

In [4]:
dataset.head(2)

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,...,10:39:55 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16 AM,12,5,...,12:17:22 PM,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993


In [5]:
riders.head(2)

,Rider Id,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Rider_Id_396,2946,2298,14.0,1159
1,Rider_Id_479,360,951,13.5,176


In [6]:
finaltest.head(2)

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Pickup - Weekday (Mo = 1),Pickup - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id
0,Order_No_19248,User_Id_3355,Bike,3,Business,27,3,4:44:10 PM,27,3,...,3,5:06:47 PM,8,NaN,NaN,-1.333275,36.870815,-1.305249,36.822390,Rider_Id_192
1,Order_No_12736,User_Id_3647,Bike,3,Business,17,5,12:57:35 PM,17,5,...,5,1:25:37 PM,5,NaN,NaN,-1.272639,36.794723,-1.277007,36.823907,Rider_Id_868


In [7]:
set(dataset.columns) - set(finaltest.columns)

{'Arrival at Destination - Day of Month',
 'Arrival at Destination - Time',
 'Arrival at Destination - Weekday (Mo = 1)',
 'Time from Pickup to Arrival'}

we must drop the above before moving on

are the columns in the final that are not on training?

In [8]:
set(finaltest.columns) - set(dataset.columns)

set()

The placement time of the order is irrelevant to the target. The main concern of the problem statement is the period between pickup and arrival. When the order was placed, has no bearing whatsoever in effect to that period. So, we are going to drop everything concerning placement. 

Furthermore, 'Confirmation' columns, with the same argument, must be dropped.

In [9]:
placement = [col for col in dataset.columns if ((col[:len('Placement')] == 'Placement') or (col[:len('Confirmation')] == 'Confirmation')) ]

In [10]:
placement

['Placement - Day of Month',
 'Placement - Weekday (Mo = 1)',
 'Placement - Time',
 'Confirmation - Day of Month',
 'Confirmation - Weekday (Mo = 1)',
 'Confirmation - Time']

In [11]:
xdata = dataset.drop(columns=['Time from Pickup to Arrival', 'User Id', 'Order No', 
'Arrival at Destination - Day of Month',
'Arrival at Destination - Time',
'Arrival at Destination - Weekday (Mo = 1)'] + placement)
X = xdata.copy() # I want to run this column everytime I want to restart after changes
X = X.merge(riders, how='left', on=['Rider Id']).drop(columns=['Rider Id'])

f_testx = finaltest.merge(riders, how='left', on=['Rider Id']).drop(columns=['User Id', 'Order No', 'Rider Id'] + placement)

In [12]:
[x for x in f_testx.columns if x[:4] in  ['Pers', 'Plat']]

['Platform Type', 'Personal or Business']

In [13]:
y = dataset['Time from Pickup to Arrival'] # The target

Confirm columns if columns on train are those on test

In [14]:
set(f_testx.columns) == set(X.columns)

True

We Split

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Now things are about to get ugly

In [16]:
### START FUNCTION
def cleaner(input_df, nullthreshold=0.9, correlation_thresh=0.95, day_of_month_cols=[], day_of_week_cols=[]):
    input_dfc = input_df.copy()
    

    #########################################################################################
    # The Code below drops columns that have null values exceeding threshold and Columns that have ONLY one value
    for col in input_df.columns:
        if (sum(input_df[col].isnull())/len(input_df[col]) > nullthreshold) or (len(input_df[col].unique()) == 1):
            input_dfc.drop(columns=[col], inplace=True) 
            
            
    #########################################################################################

    #########################################################################################
    #This code converts time given by am and pm into seconds then applies cosine and sine
    def time_to_seconds(input_df):
        input_dfc = input_df.copy()

        from datetime import datetime

        for time_col in [col for col in input_df.columns if 'Time' in [col[-4:]]]:

            input_dfc[time_col + '_sin(seconds)'] = \
            input_df[time_col].apply(lambda time: np.sin(
                (datetime.strptime(time, '%I:%M:%S %p') - datetime(1900,1,1)).total_seconds() \
                * (2.*np.pi/86400) )) # there are 86400 seconds in a day

            input_dfc[time_col + '_cos(seconds)'] = \
            input_df[time_col].apply(lambda time: np.cos(
                (datetime.strptime(time, '%I:%M:%S %p') - datetime(1900,1,1)).total_seconds() \
                * (2.*np.pi/86400) ))

            input_dfc.drop(columns=[time_col], inplace=True)

        return input_dfc

    input_dfc2 = time_to_seconds(input_dfc)
    #########################################################################################
    
    #########################################################################################
    # This code encodes ['Platform Type', 'Personal or Business']
    
    def one_encoder(input_df, columns):
    
        return pd.get_dummies(input_df, drop_first=True, columns=columns, dtype=float)

    
    input_dfc2 = one_encoder(input_dfc2, ['Platform Type', 'Personal or Business'])
    #########################################################################################
    def cyclic_days(input_df, month_days_cols, weekdays_cols):
        input_dfc = input_df.copy()


        for mday_col in month_days_cols:
            if mday_col in input_dfc.columns:

                input_dfc[mday_col + '_sin(day)'] = input_df[mday_col].apply(lambda day: np.sin(\
                day * (2.*np.pi/31))\
                    )

                input_dfc[mday_col + '_cos(day)'] = input_df[mday_col].apply(lambda day: np.cos(\
                day * (2.*np.pi/31))\
                    )

                input_dfc.drop(mday_col, inplace=True, axis=1)


        for wday_col in weekdays_cols:
            if wday_col in input_dfc.columns:

                input_dfc[wday_col + '_sin(day)'] = input_df[wday_col].apply(lambda day: np.sin(\
                day * (2.*np.pi/7))\
                    )

                input_dfc[wday_col + '_cos(day)'] = input_df[wday_col].apply(lambda day: np.cos(\
                day * (2.*np.pi/7))\
                    )

                input_dfc.drop(wday_col, inplace=True, axis=1)
            else:
                continue

        return input_dfc
    
    input_dfc2 = cyclic_days(input_dfc2, day_of_month_cols, day_of_week_cols)     
    #########################################################################################
    # This code will remove one of a pair of variables that are 95% correlated
    def correlation_drop(df, thresh):
        while True:
            corr_matrix = df.corr(method = "spearman").abs()
            upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k = 1).astype(np.bool))
            to_drop = [column for column in upper.columns if any(upper[column] > thresh)]
            if len(to_drop) == 0:
                break
            else:
                #print(to_drop) 
                df = df.drop(to_drop, axis = 1)
                print('dropped by corr', to_drop)
           
        return df
    
    input_dfc2 = correlation_drop(input_dfc2, correlation_thresh)
    #########################################################################################

    print(f"Total of {len([x for x in input_df.columns if x not in input_dfc2.columns])} original columns dropped \n")  
    print(f"Total of {len([x for x in input_dfc2.columns if x not in input_df])} new CLEAN columns formed \n")
    print(f"Dataframe now has {len(input_dfc2.columns)} from {len(input_df.columns)} input columns")
    print([x for x in input_df.columns if x not in input_dfc2.columns])

    return input_dfc2

### END FUNCTION

Now we apply our ultimate cleaner

In [17]:
day_of_month_cols = [x for x in X.columns if x[-5:] == 'Month']
day_of_week_cols = [x for x in X.columns if x[-(len('(Mo = 1)')):] == '(Mo = 1)']
X_train = cleaner(X_train, day_of_month_cols=day_of_month_cols, day_of_week_cols=day_of_week_cols)

dropped by corr ['Pickup - Time_sin(seconds)', 'Pickup - Time_cos(seconds)', 'Pickup - Day of Month_sin(day)', 'Pickup - Day of Month_cos(day)', 'Pickup - Weekday (Mo = 1)_sin(day)', 'Pickup - Weekday (Mo = 1)_cos(day)']
Total of 10 original columns dropped 

Total of 10 new CLEAN columns formed 

Dataframe now has 20 from 20 input columns
['Vehicle Type', 'Platform Type', 'Personal or Business', 'Arrival at Pickup - Day of Month', 'Arrival at Pickup - Weekday (Mo = 1)', 'Arrival at Pickup - Time', 'Pickup - Day of Month', 'Pickup - Weekday (Mo = 1)', 'Pickup - Time', 'Precipitation in millimeters']


In [31]:
day_of_week_cols

['Arrival at Pickup - Weekday (Mo = 1)', 'Pickup - Weekday (Mo = 1)']

In [19]:
X_train.head(2)

,Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Arrival at Pickup - Time_sin(seconds),Arrival at Pickup - Time_cos(seconds),Platform Type_2,Platform Type_3,Platform Type_4,Personal or Business_Personal,Arrival at Pickup - Day of Month_sin(day),Arrival at Pickup - Day of Month_cos(day),Arrival at Pickup - Weekday (Mo = 1)_sin(day),Arrival at Pickup - Weekday (Mo = 1)_cos(day)
5593,8,22.5,-1.300921,36.828195,-1.258711,36.834940,282,326,14.7,46,-0.674732,-0.738063,0.0,1.0,0.0,0.0,0.724793,0.688967,0.974928,-0.222521
6873,5,23.2,-1.260234,36.799055,-1.256810,36.773083,235,740,13.5,24,0.411846,-0.911254,0.0,1.0,0.0,0.0,0.998717,-0.050649,-0.974928,-0.222521


In [18]:
day

NameError: name 'day' is not defined

In [ ]:
X_test = cleaner(X_test, day_of_month_cols=day_of_month_cols, day_of_week_cols=day_of_week_cols)

In [20]:
X1 = cleaner(X, day_of_month_cols=day_of_month_cols, day_of_week_cols=day_of_week_cols) # When we want to create a cross validation model

dropped by corr ['Pickup - Time_sin(seconds)', 'Pickup - Time_cos(seconds)', 'Pickup - Day of Month_sin(day)', 'Pickup - Day of Month_cos(day)', 'Pickup - Weekday (Mo = 1)_sin(day)', 'Pickup - Weekday (Mo = 1)_cos(day)']
Total of 10 original columns dropped 

Total of 10 new CLEAN columns formed 

Dataframe now has 20 from 20 input columns
['Vehicle Type', 'Platform Type', 'Personal or Business', 'Arrival at Pickup - Day of Month', 'Arrival at Pickup - Weekday (Mo = 1)', 'Arrival at Pickup - Time', 'Pickup - Day of Month', 'Pickup - Weekday (Mo = 1)', 'Pickup - Time', 'Precipitation in millimeters']


We must also apply it on our finial test: 

In [ ]:
F_test = cleaner(f_testx, day_of_month_cols=day_of_month_cols, day_of_week_cols=day_of_week_cols)

In [ ]:
X1.head(2)

In [ ]:
corr_matrix = X_train.corr(method = "spearman").abs()
sns.set(font_scale = 1.0)
f, ax = plt.subplots(figsize=(11, 9))
sns.heatmap(corr_matrix, cmap= "YlGnBu", square=True, ax = ax)
f.tight_layout()


In [ ]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
polinomial_features = PolynomialFeatures(2)
# Construct the pipeline with a standard scaler and a small neural network
models = [Lasso(alpha=0.1), Ridge(), LinearRegression()]
for mod in models:
    estimators = []
    estimators.append(('imputer', SimpleImputer(strategy='median')))
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('plf', polinomial_features))
    estimators.append(('mod', mod))
    model = Pipeline(estimators)

    # To begin, let's use only these two features to predict 'cnt' (bicycle count)

    # We'll use 5-fold cross validation. That is, a random 80% of the data will be used
    # to train the model, and the prediction score will be computed on the remaining 20%.
    # This process is repeated five times such that the training sets in each "fold"
    # are mutually orthogonal.
    from sklearn.model_selection import KFold
    from sklearn.model_selection import cross_val_score
    kfold = KFold(n_splits=5)

    results = cross_val_score(model, X1, y, cv=kfold, scoring='neg_mean_squared_error')
    print(mod)
    print ('CV Scoring Result: mean=',np.sqrt(abs(np.mean(results))),'std=',np.std(results))
    print('\n')

In [ ]:
print('counts')
p1 = 0
for i in range(len(X1['Platform Type_2'])):
    if (X1['Platform Type_2'][i] == X1['Platform Type_3'][i]) == (X1['Platform Type_4'][i] == 0):
        p1 += 1

p2 = sum(X1['Platform Type_2'] == 1)
p3 = sum(X1['Platform Type_3'] == 1)
p4 = sum(X1['Platform Type_4'] == 1)

print('Platform Type_1: ', p1)
print('Platform Type_2: ', sum(X1['Platform Type_2'] == 1))
print('Platform Type_3: ', sum(X1['Platform Type_3'] == 1))
print('Platform Type_4: ', sum(X1['Platform Type_4'] == 1))


print('\n')

print('propotions')
print('total = ', sum([p1,p2,p3,p4]))
for value in [p1,p2,p3,p4]:
    print(round(value * 100/sum([p1,p2,p3,p4]), 2))
     

In [ ]:
print('counts')

personal = sum(X1['Personal or Business_Personal'] == 1)
business = sum(X1['Personal or Business_Personal'] != 1)
print('personal: ', personal, '\nbusiness: ', business, '\n')
print('propotions')
for value in [personal, business]:
    print(round(value * 100/sum([personal, business]), 2))

In [ ]:
print('counts')
p1 = 0
for i in range(len(F_test['Platform Type_2'])):
    if (F_test['Platform Type_2'][i] == F_test['Platform Type_3'][i]) == (F_test['Platform Type_4'][i] == 0):
        p1 += 1

p2 = sum(F_test['Platform Type_2'] == 1)
p3 = sum(F_test['Platform Type_3'] == 1)
p4 = sum(F_test['Platform Type_4'] == 1)

print('Platform Type_1: ', p1)
print('Platform Type_2: ', sum(F_test['Platform Type_2'] == 1))
print('Platform Type_3: ', sum(F_test['Platform Type_3'] == 1))
print('Platform Type_4: ', sum(F_test['Platform Type_4'] == 1))


print('\n')

print('propotions')
print('total = ', sum([p1,p2,p3,p4]))
for value in [p1,p2,p3,p4]:
    print(round(value * 100/sum([p1,p2,p3,p4]), 2))
     

In [ ]:
data = pd.concat([X1, y], axis=1)
data.columns

In [ ]:
data[['Temperature']].head(10)

In [ ]:
sns.boxplot(X1['Temperature'])

In [ ]:
sns.boxenplot(X1['Temperature'])

In [ ]:
print('mean: ', data['Temperature'].mean())
print('median: ', data['Temperature'].median())

In [ ]:
data['Temperature'].fillna(data['Temperature'].median(), inplace=True)

In [ ]:
corrs = data.corr()['Time from Pickup to Arrival'].sort_values(ascending=False)

from scipy.stats import pearsonr

# Build a dictionary of correlation coefficients and p-values
dict_cp = {}

column_titles = [col for col in corrs.index if col!= 'Time from Pickup to Arrival']
for col in column_titles:
    p_val = round(pearsonr(data[col], data['Time from Pickup to Arrival'])[1],6)
    dict_cp[col] = {'Correlation_Coefficient':corrs[col],
                    'P_Value':p_val}

df_cp = pd.DataFrame(dict_cp).T
df_cp_sorted = df_cp.sort_values('P_Value')
df_cp_sorted 

In [ ]:
to_drop = df_cp_sorted[df_cp_sorted['P_Value'] > 0.1].index
X_train2 = X_train.drop(columns=to_drop)
X_test2 = X_test.drop(columns=to_drop)
F_test2 = F_test.drop(columns=to_drop)
X2 = X1.drop(columns=to_drop)

In [ ]:
corr_matrix = X_train2.corr(method = "spearman").abs()
sns.set(font_scale = 1.0)
f, ax = plt.subplots(figsize=(11, 9))
sns.heatmap(corr_matrix, cmap= "YlGnBu", square=True, ax = ax)
f.tight_layout()


In [ ]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
polinomial_features = PolynomialFeatures(2)
# Construct the pipeline with a standard scaler and a small neural network
models = [Lasso(alpha=0.1), Ridge(), LinearRegression()]
for mod in models:
    estimators = []
    estimators.append(('imputer', SimpleImputer(strategy='median')))
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('plf', polinomial_features))
    estimators.append(('mod', mod))
    model = Pipeline(estimators)

    # To begin, let's use only these two features to predict 'cnt' (bicycle count)

    # We'll use 5-fold cross validation. That is, a random 80% of the data will be used
    # to train the model, and the prediction score will be computed on the remaining 20%.
    # This process is repeated five times such that the training sets in each "fold"
    # are mutually orthogonal.
    from sklearn.model_selection import KFold
    from sklearn.model_selection import cross_val_score
    kfold = KFold(n_splits=5)

    results = cross_val_score(model, X1, y, cv=kfold, scoring='neg_mean_squared_error')
    print(mod)
    print ('CV Scoring Result: mean=',np.sqrt(abs(np.mean(results))),'std=',np.std(results))
    print('\n')

Our data is ready

We will standardize the columns which have a max absolute value that is greater than 1

In [ ]:
X1.head(10)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X1)
X_normalize = pd.DataFrame(X_scaled, columns=X1.columns)

In [ ]:
X_normalize.head(3)

In [ ]:
from sklearn.feature_selection import VarianceThreshold

# Create VarianceThreshold object
selector = VarianceThreshold(threshold=0.03)

# Use the object to apply the threshold on data
selector.fit(X_normalize)

In [ ]:
# Get column variances
column_variances = selector.variances_

vars_dict = {}
vars_dict = [{"Variable_Name": c_name, "Variance": c_var}
             for c_name, c_var in zip(X_normalize.columns, column_variances)]
df_vars = pd.DataFrame(vars_dict)
df_vars.sort_values(by='Variance', ascending=False)

In [ ]:
lowvar = np.array(df_vars.loc[range(8)]['Variable_Name'])
lowvar

In [ ]:
hivar = np.array(df_vars.loc[range(8, 19)]['Variable_Name'])
hivar

In [ ]:
# Select new columns
X_new = X_normalize[X_normalize.columns[selector.get_support(indices=True)]]

# Save variable names for later
X_var_names = X_new.columns

# View first few entries
X_new.head()

In [ ]:
selector.transform(X_normalize)

In [ ]:
data.columns

In [ ]:
to_drop

In [ ]:
d = data[(data['Platform Type_4'] != 1) | (data['Platform Type_2'] != 1) | (data['Time from Pickup to Arrival'] > 60)].drop(columns=['Arrival at Pickup - Time_sin(seconds)',
       'Arrival at Pickup - Time_cos(seconds)'])
y_r_dropped = d.iloc[:, -1]
X_r_dropped = d.iloc[:, :-1]

In [ ]:
sorted(data['Time from Pickup to Arrival'])[-20:]

In [ ]:
X1.to_numpy()

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, Normalizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor, AdaBoostRegressor
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from xgboost.sklearn import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
polinomial_features = PolynomialFeatures(1)
# Construct the pipeline with a standard scaler and a small neural network
models = [CatBoostRegressor(verbose=0)]
for mod in models:
    estimators = []
    estimators.append(('imputer', SimpleImputer(strategy='median')))
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('plf', polinomial_features))
    estimators.append(('mod', mod))
    model = Pipeline(estimators)

    # To begin, let's use only these two features to predict 'cnt' (bicycle count)

    # We'll use 5-fold cross validation. That is, a random 80% of the data will be used
    # to train the model, and the prediction score will be computed on the remaining 20%.
    # This process is repeated five times such that the training sets in each "fold"
    # are mutually orthogonal.
    from sklearn.model_selection import KFold
    from sklearn.model_selection import cross_val_score
    kfold = KFold(n_splits=5)

    results = cross_val_score(model, X1.to_numpy(), y, cv=kfold, scoring='neg_mean_squared_error')
    print(mod)
    print ('CV Scoring Result: mean=',np.sqrt(abs(np.mean(results))),'std =', np.std(sorted(np.sqrt(-np.array(results)))),'list of values =',sorted(np.sqrt(-np.array(results))))
    print('\n')

CV Scoring Result: mean= 746.1454097405029 std = 21.013119269319994 list of values = [720.8251656959673, 728.2803994455254, 744.6509352073026, 755.2440619968804, 780.2467521113456]




In [25]:
m = X1.head(1)

In [26]:
m

,Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Arrival at Pickup - Time_sin(seconds),Arrival at Pickup - Time_cos(seconds),Platform Type_2,Platform Type_3,Platform Type_4,Personal or Business_Personal,Arrival at Pickup - Day of Month_sin(day),Arrival at Pickup - Day of Month_cos(day),Arrival at Pickup - Weekday (Mo = 1)_sin(day),Arrival at Pickup - Weekday (Mo = 1)_cos(day)
0,4,20.4,-1.317755,36.83037,-1.300406,36.829741,1637,1309,13.8,549,0.481817,-0.876272,0.0,1.0,0.0,0.0,0.968077,-0.250653,-0.974928,-0.222521


In [27]:
estimators = []
polinomial_features = PolynomialFeatures(1)
estimators.append(('imputer', SimpleImputer(strategy='median')))
estimators.append(('standardize', StandardScaler()))
estimators.append(('plf', polinomial_features))
estimators.append(('mod', CatBoostRegressor(verbose=0)))
model = Pipeline(estimators)

In [28]:
# v1 = sample.copy()
model.fit(X1.to_numpy(), y)
# y_pred = model.predict(F_test)
# v1['Order_No'] = finaltest['Order No']
# v1['Time from Pickup to Arrival'] = y_pred

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('standardize', StandardScaler()),
                ('plf', PolynomialFeatures(degree=1)),
                ('mod',
                 <catboost.core.CatBoostRegressor object at 0x0000020B1D239908>)])

In [29]:
y_pred = model.predict(m)

In [ ]:
v1.to_csv('catboost1.csv', index=False)

In [23]:
import pickle

model_save_path = "cat_model.pkl"
with open(model_save_path,'wb') as file:
    pickle.dump(model, file)


In [ ]:
# for x in [col for col in data.columns if col!= 'Time from Pickup to Arrival']:
#     plt.plot(x, 'Time from Pickup to Arrival', data=data)
# plt.show()   

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, Normalizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error

In [ ]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

In [ ]:
def sklearn_kfold_split(data,K):
    kf = KFold(n_splits=K, shuffle=False)

    
    return [(datatr, datate) for datatr, datate in kf.split(data)]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(data.iloc[:,:-1])
X_normalize = pd.DataFrame(X_scaled, columns=data.iloc[:,:-1].columns)

In [ ]:
X_normalize

In [ ]:
data2 = pd.concat([X_normalize, data.iloc[:,-1]], axis=1)
F_test3 = F_test.copy()

In [ ]:
ind = sklearn_kfold_split(data2, 5)

In [ ]:
def best_k_model(data,data_indices):
    models = [Lasso(alpha=0.1), LinearRegression(), RandomForestRegressor(random_state=42)]
    # polinomial_features = PolynomialFeatures(pol_f)
    d = {}
    rmses = []
    for pol_f in [1,2]:
        polinomial_features = PolynomialFeatures(pol_f)
        for mod in models:
            print('polinomial_features', polinomial_features)
            print(2*'\n', mod)
            for row in data_indices:
                estimator = []
                X_train, y_train = data[row[0], :-1], data[row[0], -1]
                X_test, y_test = data[row[1], :-1], data[row[1], -1]
                #rf = RandomForestRegressor(random_state=42)
                estimator.append(('imputer', SimpleImputer(strategy='median')))
                estimator.append(('scaler', MinMaxScaler()))
                estimator.append(('pf', polinomial_features))
                estimator.append(('model', mod))
                rf = Pipeline(estimator)
                #rf = Lasso(())
                rf.fit(X_train.reshape(-1, 1), y_train)
                mse = mean_squared_error(y_test, rf.predict(X_test.reshape(-1, 1)))
                rmse =  np.sqrt(mse)
                print (mse)
                print('rmse =', np.sqrt(mse), '\n')
                rmses.append(rmse)
                d[rmse] = rf


    print('\nbest score = ', min(rmses) , '\nby :', d[min(rmses)])
    return d[min(rmses)]

In [ ]:
data

In [ ]:
model = best_k_model(data.drop(columns=['No_Of_Orders', 'Age', 'Temperature']).values, ind)


In [ ]:
polinomial_features = PolynomialFeatures(1)
estimator = []
estimator.append(('imputer', SimpleImputer(strategy='median')))
estimator.append(('scaler', MinMaxScaler()))
estimator.append(('pf', polinomial_features))
estimator.append(('model', RandomForestRegressor(random_state=42)))
rf = Pipeline(estimator)

In [ ]:
rf.fit(X1.drop(columns=['No_Of_Orders', 'Age', 'Temperature']), y)

In [ ]:
rf.predict(F_test3.drop(columns=['No_Of_Orders', 'Age', 'Temperature']))

In [ ]:
y_pred = rf.predict(F_test3.drop(columns=['No_Of_Orders', 'Age', 'Temperature']))
v1['Order_No'] = finaltest['Order No']
v1['Time from Pickup to Arrival'] = y_pred
v1.to_csv('best_with_temp4rfsunday648pm.csv', index=False) 

In [ ]:
y_pred = model.predict(F_test3)
v1['Order_No'] = finaltest['Order No']
v1['Time from Pickup to Arrival'] = y_pred
v1.to_csv('best_without_temp3.csv', index=False)

In [ ]:
y_pred

In [ ]:
data2.values.shape

In [ ]:
model.predict(F_test3.to_numpy())

In [ ]:
set(data2.columns) - set(F_test3.columns)

In [ ]:
y_pred

In [ ]:
y_pred = model.predict(F_test3.values)

In [ ]:
model._check_n_features()

In [ ]:
model

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, Normalizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
polinomial_features = PolynomialFeatures(1)
# Construct the pipeline with a standard scaler and a small neural network
models = [Lasso(alpha=0.1), LinearRegression(), RandomForestRegressor(random_state=42)]
for mod in models:
    estimators = []
    estimators.append(('imputer', SimpleImputer(strategy='median')))
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('plf', polinomial_features))
    estimators.append(('mod', mod))
    model = Pipeline(estimators)

    # To begin, let's use only these two features to predict 'cnt' (bicycle count)

    # We'll use 5-fold cross validation. That is, a random 80% of the data will be used
    # to train the model, and the prediction score will be computed on the remaining 20%.
    # This process is repeated five times such that the training sets in each "fold"
    # are mutually orthogonal.
    from sklearn.model_selection import KFold
    from sklearn.model_selection import cross_val_score
    kfold = KFold(n_splits=5)

    results = cross_val_score(model, X1.drop(columns=['No_Of_Orders', 'Age', 'Temperature']), y, cv=kfold, scoring='neg_mean_squared_error')
    print(mod)
    print ('CV Scoring Result: mean=',np.sqrt(abs(np.mean(results))),'std=',np.std(results))
    print('\n')